In [ ]:
### Intersection of mrcanavar, PacBio CCS, and ONT high coverage regions to identify potential CNV

Generating excessive coverage bed files using calculations from mosdepth for both PacBio CCS 15kb_20kb merged and ONT bam files.

HG007vGRCh37_wm_ONT.sort.bam is from T2T Finishing Workshop globus /team-variants/read_aligns/hg005_grch37/HG007vGRCh37_wm_ONT.sort.bam

HG007.hs37d5.haplotagged_w_10x.bam is from 

Chinese-trio-HG007.hs37d5.100x.bam.bilkentuniv.052119.dups.bed.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/BilkentUni_IlluminaHiSeq_TARDIS_mrCaNaVar_05212019/Chinese-trio-HG007.hs37d5.100x.bam.bilkentuniv.052119.dups.bed.gz

HG007.hs37d5.pbsv.vcf.gz is from https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/ChineseTrio/analysis/PacBio_deepvariant_gatk_pbsv_08282020/HG007/HG007.hs37d5.pbsv.vcf.gz


convert_mosdepth_to_excessive_coverage.py is at the end of this notebook

In [ ]:
### mosdepth commands

In [ ]:
mosdepth -b 1000 --mapq 20 --no-per-base HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20 HG007.hs37d5.haplotagged.bam

samtools view -1 -F 0x100 HG007vGRCh37_wm_ONT.sort.bam -h > HG007vGRCh37_wm_ONT.sort_filtered.bam

mosdepth -b 1000 -x --no-per-base HG007_GRCh37_ONT_filtered_window_size HG007vGRCh37_wm_ONT.sort_filtered.bam

In [ ]:
### Find coverage levels of excessive coverage on CCS and ONT data in R

In [ ]:
chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")
mosdepth_PB_HiFi_1000_window_size = read.delim("HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_PB_HiFi_1000_window_size_chr_1_22 <- mosdepth_PB_HiFi_1000_window_size[which(mosdepth_PB_HiFi_1000_window_size[,"CHR"] %in% chr_1_22),] 
quantile(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])


IQR(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])
#: 15.9
(quantile(mosdepth_PB_HiFi_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 71.5125 

mosdepth_ONT_1000_window_size = read.delim("HG007_GRCh37_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))
mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])

IQR(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])
#: 10.35
(quantile(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"])[3]/2)*2.5
#: 50.85

In [ ]:
### GRCh37_HG007_GIABv4.2.1_CNV_mrcanavarIllumina_CCShighcov_ONThighcov_intersection.bed

### What this does: find potential CNVs in HG7 from intersecting coverage files from PacBio HiFi, ONT, and Illumina data. This generates PacBio HiFi excessive coverage bed, intersect with mrCaNaVar dups bed, generates ONT excessive coverage bed, intersects to these all to generate exclusion bed

In [ ]:
python convert_mosdepth_to_excessive_coverage.py --input HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20.regions.bed --output HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20_excessive_coverage_cnv_threshold.bed --threshold 71.5125 

bedtools intersect -a Chinese-trio-HG007.hs37d5.100x.bam.bilkentuniv.052119.dups.bed -b HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_PacBio_HiFi_cnv_threshold.bed

python convert_mosdepth_to_excessive_coverage.py --input HG007_GRCh37_ONT_filtered_window_size.regions.bed --output HG007_GRCh37_ONT_filtered_window_size_excessive_coverage_cnv_threshold.bed --threshold 50.85

bedtools intersect -a mrcanavar_intersect_PacBio_HiFi_cnv_threshold.bed -b HG007_GRCh37_ONT_filtered_window_size_excessive_coverage_cnv_threshold.bed > mrcanavar_intersect_PacBio_HiFi_cnv_threshold_intersect_ONT_cnv_threshold.bed

cp mrcanavar_intersect_PacBio_HiFi_cnv_threshold_intersect_ONT_cnv_threshold.bed GRCh37_HG007_GIABv4.2.1_CNV_mrcanavarIllumina_CCShighcov_ONThighcov_intersection.bed


In [ ]:
### HG007_GRCh37_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_outlier_threshold.bed

### What this does: Find another set of potential CNVs from computing a coverage threshold using an elliciptal outlier for PacBio HiFi and ONT CNV file steps in R

In [ ]:
mosdepth_PacBio_HiFi_1000_window_size = read.delim("HG007_GRCh37_PacBio_HiFi_1000_window_size_MAPQ20.regions.bed", col.names = c("CHR","START","END","DEPTH"))

mosdepth_ONT_1000_window_size = read.delim("HG007_GRCh37_ONT_filtered_window_size.regions.bed", col.names = c("CHR","START","END","DEPTH"))

chr_1_22 <- c("1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22")

mosdepth_PacBio_HiFi_1000_window_size_chr_1_22 <- mosdepth_PacBio_HiFi_1000_window_size[which(mosdepth_PacBio_HiFi_1000_window_size[,"CHR"] %in% chr_1_22),]

mosdepth_ONT_1000_window_size_chr_1_22 <- mosdepth_ONT_1000_window_size[which(mosdepth_ONT_1000_window_size[,"CHR"] %in% chr_1_22),]

df_mosdepth_PacBio_HiFi_1000_window_size <- data.frame(mosdepth_PacBio_HiFi_1000_window_size_chr_1_22) 

df_mosdepth_ONT_1000_window_size <- data.frame(mosdepth_ONT_1000_window_size_chr_1_22)

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined <- df_mosdepth_PacBio_HiFi_1000_window_size

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,5] <- mosdepth_ONT_1000_window_size_chr_1_22[,4]

colnames(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined) <- c("CHR", "START", "END", "CCS_DEPTH", "ONT_DEPTH")

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values <- df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined 

medianccsdepth = median(mosdepth_PacBio_HiFi_1000_window_size_chr_1_22[,"DEPTH"]) 

medianontdepth = median(mosdepth_ONT_1000_window_size_chr_1_22[,"DEPTH"]) 

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6] <- sqrt(((df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,4]/medianccsdepth)^2 + (df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined[,5]/medianontdepth)^2)/2) 

threshold_ellipctial_outlier = unname(quantile(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6])[4]+(1.5*IQR(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6])[1]))

df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_outliers <- df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[which(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_values[,6] > threshold_ellipctial_outlier),]

write.csv(df_mosdepth_PacBio_HiFi_1000_window_size_ONT_1000_combined_outliers, file = "HG007_GRCh37_PacBio_HiFi_1000_window_size_ONT_1000_combined_elliptical_outlier_threshold.bed", row.names = FALSE)


In [ ]:
### pbsv_HG007.GRCh37_gt49bp_slop50_repeatexpanded_slop25percent.bed
### What this does: find SV calls from PacBio HiFi that will be excluded from HG7 in addition to the v0.6 GIAB SV calls

In [ ]:
#Form bed to exclude SV regions from HG003 v4.2.1 benchmark

zgrep -v ^# HG007.hs37d5.pbsv.vcf.gz | awk '{FS=OFS="\t"} {  if(length($4)>49 || length($5)>49) print $1,$2-50,$2+length($4)+50} ' > pbsv_HG007.GRCh37_gt49bp_slop50.bed

cat  pbsv_HG007.GRCh37_gt49bp_slop50.bed | sed 's/^X/23/;s/^Y/24/'| grep -Ev '^M|^[0-9][0-9]_|^[0-9]_|^[0-9]\||^[0-9][0-9]\||^Un|^HS' | sort -k1,1n -k2,2n -k3,3n | sed 's/^23/X/;s/^24/Y/' >  pbsv_HG007.GRCh37_gt49bp_slop50_sorted.bed

# pbsv_HG007.GRCh37_gt49bp_slop50_repeatexpanded_slop25percent.bed
intersectBed -wa -a GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz -b pbsv_HG007.GRCh37_gt49bp_slop50_sorted.bed | multiIntersectBed -i stdin pbsv_HG007.GRCh37_gt49bp_slop50_sorted.bed | mergeBed -i stdin -d 1000 | awk '{FS=OFS="\t"} { slop=int(0.25*($3-$2)); if(slop>5000) slop=5000; print $1, $2-slop, $3+slop}' | awk '{FS=OFS="\t"} { if($2<0) $2=0; print}' > GRCh37_HG007_GIABv4.2.1_SV_pbsv_slop25percent

In [ ]:
#### convert_mosdepth_to_excessive_coverage.py script

import argparse

parser = argparse.ArgumentParser(description="Subset bed file to callable regions only")
parser.add_argument('--input_file', metavar="I", type=str, nargs="+", help="input bed file")
parser.add_argument('--output_file', metavar="O", type=str, nargs="+", help="output file")
parser.add_argument('--threshold', metavar="T", type=str, nargs="+", help="input threshold")
args = parser.parse_args()

f = open(args.input_file[0], "r") 
f_lines = f.readlines()

f_out = open(args.output_file[0], "w+")
threshold = float(args.threshold[0])

for line in f_lines:   
    if "DEPTH" in line: 
        continue
    line_split = line.split("\t")
    depth_field = float(line_split[3])
    if depth_field > threshold:
        f_out.write(line)
        f_out.flush()  

f.close()
f_out.close()